In [1]:
pip install earthengine-api

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee

# Authenticate to access Google Earth Engine
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize()


Enter verification code: 4/1AVG7fiQ8i2roS7IAxoOSgW3GvAnwu0lQ9s5VcWrrkp0yRjQVimFqhd9T2po

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [6]:
import ee

# Initialize Earth Engine (after successful authentication)
ee.Initialize()

# Corrected dataset ID for Landsat 8 Surface Reflectance Tier 1
LANDSAT_SR = 'LANDSAT/LC08/C02/T1_L2'

# Function to retrieve Landsat Surface Reflectance (SR) data
def get_landsat_sr_data(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    """
    Retrieve Landsat Surface Reflectance (SR) data for a given location and date range,
    applying a cloud cover filter.
    
    Parameters:
    lat (float): Latitude of the location
    lon (float): Longitude of the location
    start_date (str): Start date for the data collection (YYYY-MM-DD)
    end_date (str): End date for the data collection (YYYY-MM-DD)
    cloud_cover_threshold (int): Maximum allowed cloud cover percentage (default 20%)
    
    Returns:
    tuple: A tuple containing metadata and a thumbnail URL for visualization
    """
    # Define the point of interest (AOI)
    point = ee.Geometry.Point([lon, lat])
    
    # Access Landsat 8 SR data (using correct dataset ID)
    landsat_sr = ee.ImageCollection(LANDSAT_SR) \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .filterMetadata('CLOUD_COVER', 'less_than', cloud_cover_threshold)
    
    # Select relevant bands (Surface Reflectance Bands)
    sr_bands = landsat_sr.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).median()
    
    # Create visualization parameters for RGB bands
    vis_params = {
        'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # RGB bands
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }
    
    # Get a thumbnail URL for quick visualization
    thumbnail_url = sr_bands.getThumbURL({
        'region': point.buffer(5000).bounds().getInfo()['coordinates'],  # 5km buffer around the point
        'dimensions': 512,
        'format': 'png',
        'min': 0,
        'max': 3000,
        'bands': ['SR_B4', 'SR_B3', 'SR_B2']
    })
    
    # Get image metadata (IDs of images in the collection)
    metadata = landsat_sr.aggregate_array('system:id').getInfo()

    return metadata, thumbnail_url

# Example usage: Retrieve Landsat SR data for Bangalore (lat, lon), within the year 2020
latitude = 12.9716   # Bangalore Latitude
longitude = 77.5946  # Bangalore Longitude
start_date = '2020-01-01'
end_date = '2020-12-31'
cloud_cover_threshold = 10  # Filter by less than 10% cloud cover

# Get Landsat data for the specified location and date range
metadata, image_url = get_landsat_sr_data(latitude, longitude, start_date, end_date, cloud_cover_threshold)

# Print metadata (image IDs) and the thumbnail URL for visualization
print("Metadata (Image IDs):", metadata)
print("Thumbnail URL:", image_url)


Metadata (Image IDs): ['LANDSAT/LC08/C02/T1_L2/LC08_144051_20200111', 'LANDSAT/LC08/C02/T1_L2/LC08_144051_20200127', 'LANDSAT/LC08/C02/T1_L2/LC08_144051_20200212', 'LANDSAT/LC08/C02/T1_L2/LC08_144051_20200315', 'LANDSAT/LC08/C02/T1_L2/LC08_144051_20200331', 'LANDSAT/LC08/C02/T1_L2/LC08_144051_20200416', 'LANDSAT/LC08/C02/T1_L2/LC08_144051_20200502', 'LANDSAT/LC08/C02/T1_L2/LC08_144051_20201110', 'LANDSAT/LC08/C02/T1_L2/LC08_144051_20201212']
Thumbnail URL: https://earthengine.googleapis.com/v1/projects/966546622171/thumbnails/6d944403ad8d2a8fb45d58ede5cb81d5-f0e785ef663a647119146c2cb99e143d:getPixels


In [15]:
def user_input():
    """
    Collects input from the user for target location, date range, and cloud cover.
    """
    lat = float(input("Enter the latitude of the location: "))
    lon = float(input("Enter the longitude of the location: "))
    start_date = input("Enter the start date (YYYY-MM-DD): ")
    end_date = input("Enter the end date (YYYY-MM-DD): ")
    cloud_cover = int(input("Enter the maximum cloud cover percentage (0-100): "))
    
    return lat, lon, start_date, end_date, cloud_cover

# Example to get user input
latitude, longitude, start_date, end_date, cloud_cover_threshold = user_input()


Enter the latitude of the location: 12.9716
Enter the longitude of the location: 77.5946
Enter the start date (YYYY-MM-DD): 2020-01-01
Enter the end date (YYYY-MM-DD): 2020-12-31
Enter the maximum cloud cover percentage (0-100): 20


In [17]:
def get_landsat_thumbnail_url(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    try:
        # Define the point of interest (AOI)
        point = ee.Geometry.Point([lon, lat])

        # Access Landsat 8 SR data
        landsat_sr = ee.ImageCollection(LANDSAT_SR) \
                        .filterBounds(point) \
                        .filterDate(start_date, end_date) \
                        .filterMetadata('CLOUD_COVER', 'less_than', cloud_cover_threshold)
        
        # Select relevant bands (Surface Reflectance Bands)
        sr_bands = landsat_sr.select(['SR_B4', 'SR_B3', 'SR_B2']).median()

        # Create visualization parameters
        vis_params = {
            'min': 0,
            'max': 3000,
            'gamma': 1.4
        }

        # Generate thumbnail URL
        thumbnail_url = sr_bands.getThumbURL({
            'region': point.buffer(5000).bounds().getInfo()['coordinates'],  # 5km buffer
            'dimensions': 512,
            'format': 'png',
            'min': 0,
            'max': 3000,
            'bands': ['SR_B4', 'SR_B3', 'SR_B2']
        })

        return thumbnail_url
    except Exception as e:
        print(f"Error generating thumbnail URL: {e}")
        return None

# Example usage
image_url = get_landsat_thumbnail_url(latitude, longitude, start_date, end_date, cloud_cover_threshold)

# Check if URL is generated successfully
if image_url:
    print(f"Thumbnail URL: {image_url}")
    display(Image(url=image_url))
else:
    print("Failed to generate image thumbnail.")


Thumbnail URL: https://earthengine.googleapis.com/v1/projects/966546622171/thumbnails/46cfc65a54d3dfafb2b7cc991800801a-988fbc8ba2d949b9d32fee4466736355:getPixels


In [19]:
import requests
from PIL import Image as PILImage
from io import BytesIO

# Download the image from URL
response = requests.get(image_url)
if response.status_code == 200:
    img = PILImage.open(BytesIO(response.content))
    img.show()
else:
    print("Failed to retrieve the image from the URL.")


In [21]:
def get_landsat_sr_data(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    """
    Retrieve Landsat Surface Reflectance (SR) data for a given location and date range,
    applying a cloud cover filter.
    """
    # Define the point of interest (AOI)
    point = ee.Geometry.Point([lon, lat])
    
    # Access Landsat 8 SR data (using correct dataset ID)
    landsat_sr = ee.ImageCollection(LANDSAT_SR) \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .filterMetadata('CLOUD_COVER', 'less_than', cloud_cover_threshold)
    
    # Check if there are any images in the collection
    count = landsat_sr.size().getInfo()
    print(f"Number of images found: {count}")
    
    if count == 0:
        print("No images found for the specified region and date range.")
        return None, None
    
    # Select relevant bands (Surface Reflectance Bands)
    sr_bands = landsat_sr.select(['SR_B4', 'SR_B3', 'SR_B2']).median()
    
    # Create visualization parameters for RGB bands
    vis_params = {
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }
    
    # Get a thumbnail URL for visualization
    thumbnail_url = sr_bands.getThumbURL({
        'region': point.buffer(5000).bounds().getInfo()['coordinates'],  # 5km buffer around the point
        'dimensions': 512,
        'format': 'png',
        'min': 0,
        'max': 3000,
        'bands': ['SR_B4', 'SR_B3', 'SR_B2']
    })
    
    return landsat_sr, thumbnail_url

# Example usage
latitude = 12.9716   # Bangalore Latitude
longitude = 77.5946  # Bangalore Longitude
start_date = '2020-01-01'
end_date = '2020-12-31'
cloud_cover_threshold = 50  # Filter by less than 10% cloud cover

# Get Landsat data for the specified location and date range
metadata, image_url = get_landsat_sr_data(latitude, longitude, start_date, end_date, cloud_cover_threshold)

# If images are found, display them
if image_url:
    print(f"Thumbnail URL: {image_url}")
    from IPython.display import Image, display
    display(Image(url=image_url))
else:
    print("No images available for the selected criteria.")


Number of images found: 17
Thumbnail URL: https://earthengine.googleapis.com/v1/projects/966546622171/thumbnails/d8f33d7fde3d1ff43e9226aa80d09d3e-87a772fe602b4f7850eca8b84cab9b4a:getPixels


In [29]:
pip install earthengine-api --upgrade

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ee
ee.Authenticate()  # This will prompt for authentication
ee.Initialize()


In [3]:
import ee

# Initialize Earth Engine
ee.Initialize()

def get_landsat_sr_data(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    # Define the point of interest
    point = ee.Geometry.Point([lon, lat])

    # Access Landsat 8 SR data
    landsat_sr = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .filterMetadata('CLOUD_COVER', 'less_than', cloud_cover_threshold)

    # Check if there are any images in the collection
    count = landsat_sr.size().getInfo()
    print(f"Number of images found: {count}")

    if count == 0:
        print("No images found for the specified region and date range.")
        return None, None

    # Select the Surface Reflectance bands
    sr_bands = landsat_sr.select(['SR_B4', 'SR_B3', 'SR_B2']).median()
    
    # Create visualization parameters
    thumbnail_url = sr_bands.getThumbURL({
        'region': point.buffer(5000).bounds().getInfo()['coordinates'],
        'dimensions': 512,
        'format': 'png',
        'min': 0,
        'max': 3000,
        'bands': ['SR_B4', 'SR_B3', 'SR_B2']
    })

    return landsat_sr, thumbnail_url

# Example usage
latitude = 12.9716   # Bangalore Latitude
longitude = 77.5946  # Bangalore Longitude
start_date = '2020-01-01'
end_date = '2020-12-31'
cloud_cover_threshold = 20  # Filter by less than 20% cloud cover

# Get Landsat data
metadata, image_url = get_landsat_sr_data(latitude, longitude, start_date, end_date, cloud_cover_threshold)

# If images are found, display the thumbnail
if image_url:
    from IPython.display import Image, display
    display(Image(url=image_url))
else:
    print("No images available for the selected criteria.")


/Users/poojyanthm/anaconda3/lib/python3.10/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for LANDSAT/LC08/C01/T1_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/landsat_c1_to_c2

  warnings.warn(warning, category=DeprecationWarning)


EEException: ImageCollection.load: ImageCollection asset 'LANDSAT/LC08/C01/T1_SR' not found (does not exist or caller does not have access).

In [12]:
pip install widgetsnbextension


Note: you may need to restart the kernel to use updated packages.


In [4]:
import ee
import json
from IPython.display import Image, display, HTML

# Initialize Earth Engine
ee.Authenticate()  # Authenticate with your personal account
ee.Initialize()

def get_landsat_sr_data(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    point = ee.Geometry.Point([lon, lat])

    # Access Landsat 8 SR data
    landsat_sr = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloud_cover_threshold)

    count = landsat_sr.size().getInfo()
    print(f"Number of images found: {count}")

    if count == 0:
        print("No images found for the specified region and date range.")
        return None, None, None

    image = landsat_sr.first()
    metadata = image.getInfo()
    image_url = image.getThumbURL({'region': point.buffer(5000).bounds().getInfo()['coordinates'],
                                     'dimensions': 512,
                                     'format': 'png',
                                     'min': 0,
                                     'max': 3000,
                                     'bands': ['B4', 'B3', 'B2']})

    return metadata, image_url

# Example user input for testing
lat = 12.9716
lon = 77.5946
start_date = "2020-01-01"
end_date = "2020-12-31"

# Retrieve and display image and metadata
metadata, image_url = get_landsat_sr_data(lat, lon, start_date, end_date)
if image_url:
    display(Image(url=image_url))
    print(metadata)


EEException: ImageCollection.load: ImageCollection asset 'LANDSAT/LC08/C01/T1_SR' not found (does not exist or caller does not have access).

In [5]:
import ee

# Initialize Earth Engine for your personal account
ee.Authenticate()  # Log in with your personal account
ee.Initialize()

# Try retrieving Landsat data again
metadata, image_url = get_landsat_sr_data(lat, lon, start_date, end_date)


EEException: ImageCollection.load: ImageCollection asset 'LANDSAT/LC08/C01/T1_SR' not found (does not exist or caller does not have access).

In [13]:
import ee

# Initialize Earth Engine
ee.Authenticate()  # Authenticate with your account
ee.Initialize()

def test_sentinel_2_access(lat, lon, start_date, end_date):
    point = ee.Geometry.Point([lon, lat])

    # Use Sentinel-2 data
    sentinel_sr = ee.ImageCollection("COPERNICUS/S2") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date)

    count = sentinel_sr.size().getInfo()
    print(f"Number of images found for Sentinel-2: {count}")

# Example user input for testing
lat = 12.9716  # Latitude for Bangalore, India
lon = 77.5946  # Longitude for Bangalore, India
start_date = "2015-01-01"  # Start date
end_date = "2023-01-01"    # End date

# Test Sentinel-2 access
test_sentinel_2_access(lat, lon, start_date, end_date)


/Users/poojyanthm/anaconda3/lib/python3.10/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Number of images found for Sentinel-2: 477


In [14]:
import ee
from IPython.display import Image, display, HTML

# Initialize Earth Engine
ee.Authenticate()  # Authenticate with your account
ee.Initialize()

def retrieve_and_display_sentinel_image(lat, lon, start_date, end_date):
    point = ee.Geometry.Point([lon, lat])

    # Use Sentinel-2 data
    sentinel_sr = ee.ImageCollection("COPERNICUS/S2") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .sort('system:time_start')  # Sort by date

    count = sentinel_sr.size().getInfo()
    print(f"Number of images found: {count}")

    if count == 0:
        print("No images found for the specified region and date range.")
        return

    # Get the first image from the collection
    image = sentinel_sr.first()
    metadata = image.getInfo()

    # Print the metadata to check its structure
    print("Image Metadata:")
    print(json.dumps(metadata, indent=2))

    # Generate thumbnail URL for the image
    image_url = image.getThumbURL({
        'region': point.buffer(5000).bounds().getInfo()['coordinates'],
        'dimensions': 512,
        'format': 'png',
        'min': 0,
        'max': 3000,
        'bands': ['B4', 'B3', 'B2']  # RGB bands
    })

    # Display the image thumbnail
    print(f"Image URL: {image_url}")
    display(Image(url=image_url))

    # Display metadata information
    acquisition_date = metadata['properties'].get('system:time_start', 'N/A')
    cloud_cover = metadata['properties'].get('system:cloud_coverage', 'N/A')
    image_id = metadata.get('id', 'N/A')

    metadata_info = f"""
    <h4>Image Metadata:</h4>
    <ul>
        <li><strong>Acquisition Date:</strong> {acquisition_date}</li>
        <li><strong>Cloud Cover (%):</strong> {cloud_cover}</li>
        <li><strong>Image ID:</strong> {image_id}</li>
    </ul>
    """
    display(HTML(metadata_info))

# Example user input for testing
lat = 12.9716  # Latitude for Bangalore, India
lon = 77.5946  # Longitude for Bangalore, India
start_date = "2015-01-01"  # Start date
end_date = "2023-01-01"    # End date

# Retrieve and display Sentinel-2 image and metadata
retrieve_and_display_sentinel_image(lat, lon, start_date, end_date)


Number of images found: 477
Image Metadata:
{
  "type": "Image",
  "bands": [
    {
      "id": "B1",
      "data_type": {
        "type": "PixelType",
        "precision": "int",
        "min": 0,
        "max": 65535
      },
      "dimensions": [
        1830,
        1830
      ],
      "crs": "EPSG:32643",
      "crs_transform": [
        60,
        0,
        699960,
        0,
        -60,
        1500000
      ]
    },
    {
      "id": "B2",
      "data_type": {
        "type": "PixelType",
        "precision": "int",
        "min": 0,
        "max": 65535
      },
      "dimensions": [
        10980,
        10980
      ],
      "crs": "EPSG:32643",
      "crs_transform": [
        10,
        0,
        699960,
        0,
        -10,
        1500000
      ]
    },
    {
      "id": "B3",
      "data_type": {
        "type": "PixelType",
        "precision": "int",
        "min": 0,
        "max": 65535
      },
      "dimensions": [
        10980,
        10980
      ],
  

In [1]:
# Import necessary libraries
import ee
from IPython.display import Image, display
from datetime import datetime

# Initialize Earth Engine
ee.Authenticate()  # Authenticate with your personal account
ee.Initialize()

def get_landsat_sr_data(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    """Retrieve Landsat Surface Reflectance (SR) data for a given location and date range."""
    point = ee.Geometry.Point([lon, lat])
    
    # Access Landsat 8 SR data
    landsat_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloud_cover_threshold)

    count = landsat_sr.size().getInfo()
    if count == 0:
        print("No Landsat images found for the specified region and date range.")
        return None, None
    
    # Select the Surface Reflectance bands
    sr_bands = landsat_sr.select(['SR_B4', 'SR_B3', 'SR_B2']).median()
    
    # Generate thumbnail URL
    thumbnail_url = sr_bands.getThumbURL({
        'region': point.buffer(5000).bounds().getInfo()['coordinates'],
        'dimensions': 512,
        'format': 'png',
        'min': 0,
        'max': 3000,
        'bands': ['SR_B4', 'SR_B3', 'SR_B2']
    })

    return landsat_sr, thumbnail_url

def get_sentinel_sr_data(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    """Retrieve Sentinel-2 data for a given location and date range."""
    point = ee.Geometry.Point([lon, lat])
    
    sentinel_sr = ee.ImageCollection("COPERNICUS/S2") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloud_cover_threshold)

    count = sentinel_sr.size().getInfo()
    if count == 0:
        print("No Sentinel images found for the specified region and date range.")
        return None, None
    
    # Get the first image from the collection
    image = sentinel_sr.first()
    image_url = image.getThumbURL({
        'region': point.buffer(5000).bounds().getInfo()['coordinates'],
        'dimensions': 512,
        'format': 'png',
        'min': 0,
        'max': 3000,
        'bands': ['B4', 'B3', 'B2']
    })

    return sentinel_sr, image_url

def retrieve_and_display_data(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    """Retrieve and display data from both Landsat and Sentinel datasets."""
    # Landsat data retrieval
    landsat_data, landsat_thumbnail = get_landsat_sr_data(lat, lon, start_date, end_date, cloud_cover_threshold)
    if landsat_thumbnail:
        print("Landsat Thumbnail URL:", landsat_thumbnail)
        display(Image(url=landsat_thumbnail))

    # Sentinel data retrieval
    sentinel_data, sentinel_thumbnail = get_sentinel_sr_data(lat, lon, start_date, end_date, cloud_cover_threshold)
    if sentinel_thumbnail:
        print("Sentinel Thumbnail URL:", sentinel_thumbnail)
        display(Image(url=sentinel_thumbnail))

# Main Execution Block
if __name__ == "__main__":
    # Hardcoded input parameters
    latitude = 12.9716      # Example: Latitude for Bangalore
    longitude = 77.5946     # Example: Longitude for Bangalore
    start_date = '2023-01-01'  # Example start date
    end_date = '2023-12-31'    # Example end date
    cloud_cover_threshold = 20  # Example maximum cloud cover percentage

    # Retrieve and display data using hardcoded parameters
    retrieve_and_display_data(latitude, longitude, start_date, end_date, cloud_cover_threshold)


No Landsat images found for the specified region and date range.


/Users/poojyanthm/anaconda3/lib/python3.10/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Sentinel Thumbnail URL: https://earthengine.googleapis.com/v1/projects/966546622171/thumbnails/2532387c83798cd89b40e9636ebed2f6-2561855c7024334f20a7a59678dbf438:getPixels


In [10]:
import ee
ee.Authenticate()
ee.Initialize()


In [15]:
import ee
import time  # Importing the time module

# Initialize the Earth Engine module
ee.Initialize()

def check_image_count(lat, lon, start_date, end_date):
    """Check how many Landsat images are available in the specified date range."""
    point = ee.Geometry.Point([lon, lat])
    landsat_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date)

    count = landsat_sr.size().getInfo()  # Get the number of images
    print("Number of images found:", count)  # Debugging output
    return count

def export_landsat_image(lat, lon, start_date, end_date, cloud_cover_threshold, export_bucket):
    """Export the Landsat image to Google Cloud Storage."""
    point = ee.Geometry.Point([lon, lat])
    landsat_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover_threshold)) \
                    .first()  # Get the first image after filtering

    if landsat_sr is None:
        print("No Landsat image available for the given parameters.")  # Debugging output
        return

    # Export the image
    task = ee.batch.Export.image.toCloudStorage(
        image=landsat_sr.select('SR_B4', 'SR_B3', 'SR_B2'),  # Select bands for RGB
        description='LandsatImageExport',
        bucket=export_bucket,
        fileNamePrefix='landsat_image',
        region=point.buffer(1000).bounds(),  # Define export region
        scale=30,  # Scale in meters
        maxPixels=1e13
    )

    task.start()  # Start the export task
    print("Export task started. Check your Google Cloud Storage for results.")  # Debugging output

    # Check the status of the task
    while task.active():
        print('Task status:', task.status())  # Print task status
        time.sleep(10)  # Wait for a while before checking again

def prepare_metadata(lat, lon, start_date, end_date):
    """Prepare metadata for the Landsat image."""
    point = ee.Geometry.Point([lon, lat])
    landsat_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                    .filterBounds(point) \
                    .filterDate(start_date, end_date) \
                    .first()

    if landsat_sr is None:
        print("No Landsat metadata available for the given parameters.")  # Debugging output
        return None

    metadata = {
        'path': landsat_sr.get('system:index').getInfo(),
        'row': landsat_sr.get('system:footprint').getInfo(),  # Get footprint as metadata
        'cloud_cover': landsat_sr.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    }
    print("Metadata retrieved:", metadata)  # Debugging output
    return metadata

def main(lat, lon, start_date, end_date, cloud_cover_threshold=20, export_bucket='your-gcs-bucket'):
    print("Checking image availability...")  # Debugging output
    image_count = check_image_count(lat, lon, start_date, end_date)

    if image_count > 0:  # Proceed only if there are images
        print("Exporting Landsat image...")  # Debugging output
        export_landsat_image(lat, lon, start_date, end_date, cloud_cover_threshold, export_bucket)

        print("Preparing metadata...")  # Debugging output
        metadata = prepare_metadata(lat, lon, start_date, end_date)
        if metadata:
            print("Metadata prepared:", metadata)
        else:
            print("No metadata found.")
    else:
        print("No images available in the specified date range.")

# Example usage
lat = 15.3173  # Replace with your latitude
lon = 75.7139  # Replace with your longitude
start_date = '2023-01-01'  # Replace with your start date
end_date = '2023-12-31'    # Replace with your end date
export_bucket = 'your-gcs-bucket'  # Replace with your Google Cloud Storage bucket name

main(lat, lon, start_date, end_date, cloud_cover_threshold=20, export_bucket=export_bucket)


Checking image availability...
Number of images found: 42
Exporting Landsat image...
Export task started. Check your Google Cloud Storage for results.
Task status: {'state': 'FAILED', 'description': 'LandsatImageExport', 'priority': 100, 'creation_timestamp_ms': 1728195106103, 'update_timestamp_ms': 1728195108635, 'start_timestamp_ms': 1728195108357, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Image.select: Parameter 'input' is required.", 'id': 'YP37AL5CP3ESAVIDGMRFUI5L', 'name': 'projects/966546622171/operations/YP37AL5CP3ESAVIDGMRFUI5L'}
Preparing metadata...
Metadata retrieved: {'path': 'LC08_145049_20230110', 'row': {'type': 'LinearRing', 'coordinates': [[76.79225224356185, 14.89832478372192], [76.90634156737481, 14.873817359730094], [77.00947455135173, 14.851618612983488], [77.00992208541714, 14.852886429344036], [77.04786973563408, 15.019953143331978], [77.10339392161612, 15.264280167007492], [77.12092396585835, 15.341369509315316], [77.17713464209255, 15.588302

In [18]:
import ee
import json

# Initialize the Earth Engine API
ee.Initialize()

# Define the Sentinel-2 dataset ID
SENTINEL_2_SR = 'COPERNICUS/S2'

# Function to retrieve Sentinel-2 Surface Reflectance (SR) data and export as GeoTIFF
def export_sentinel2_sr(lat, lon, start_date, end_date, cloud_cover_threshold=20):
    # Define the point of interest
    point = ee.Geometry.Point([lon, lat])

    # Access Sentinel-2 SR data
    try:
        sentinel_sr = ee.ImageCollection(SENTINEL_2_SR) \
                        .filterBounds(point) \
                        .filterDate(start_date, end_date) \
                        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloud_cover_threshold)

        count = sentinel_sr.size().getInfo()
        print(f"Number of images found: {count}")

        if count == 0:
            print("No images found for the specified region and date range.")
            return None, None

        # Select the first image
        image = sentinel_sr.first()
        
        # Define the export task
        task = ee.batch.Export.image.toDrive(
            image=image,
            description='Sentinel2_SR_Export',
            scale=10,  # Sentinel-2 has a 10m resolution for certain bands
            region=point.buffer(5000).bounds().getInfo()['coordinates'],
            fileFormat='GeoTIFF'
        )

        # Start the export task
        task.start()
        print("Export task started. Check your Google Drive for results.")

        # Prepare metadata
        metadata = {
            "system:index": image.get('system:index').getInfo(),
            "system:time_start": image.get('system:time_start').getInfo(),
            "system:cloud_coverage": image.get('system:cloud_coverage').getInfo(),
            "CLOUDY_PIXEL_PERCENTAGE": image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo(),
            "bands": image.bandNames().getInfo()
        }

        # Convert metadata to JSON
        metadata_json = json.dumps(metadata)
        
        return task, metadata_json

    except Exception as e:
        print(f"Error accessing Sentinel-2 SR data: {e}")
        return None, None

# Example usage
latitude = 12.9716   # Bangalore Latitude
longitude = 77.5946  # Bangalore Longitude
start_date = '2020-01-01'
end_date = '2020-12-31'
cloud_cover_threshold = 20  # Filter by less than 20% cloud cover

# Export Sentinel-2 data and retrieve metadata
export_task, metadata_json = export_sentinel2_sr(latitude, longitude, start_date, end_date, cloud_cover_threshold)

# Print the metadata JSON
if metadata_json:
    print("Metadata JSON:", metadata_json)


Number of images found: 27
Export task started. Check your Google Drive for results.
Metadata JSON: {"system:index": "20200104T051209_20200104T051616_T43PGQ", "system:time_start": 1578115514759, "system:cloud_coverage": null, "CLOUDY_PIXEL_PERCENTAGE": 10.006, "bands": ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12", "QA10", "QA20", "QA60"]}


In [19]:
// Define your area of interest (AOI) as a rectangular region
var aoi = ee.Geometry.Rectangle([77.5, 13.0, 78.0, 13.5]); // Example coordinates for a rectangular area

// Define the time range for the data
var startDate = '2021-01-01';
var endDate = '2021-12-31';

// Load the Sentinel-2 surface reflectance image collection
var s2Collection = ee.ImageCollection('COPERNICUS/S2')
                    .filterBounds(aoi)
                    .filterDate(startDate, endDate)
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)); // Cloud cover < 20%

// Function to extract spectral signature
var getSpectralSignature = function(image) {
  var bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']; // Selected bands
  var mean = image.reduceRegion({
    reducer: ee.Reducer.mean(),
    geometry: aoi,
    scale: 10,
    bestEffort: true
  });
  
  return ee.Feature(null, {
    'B2': mean.get('B2'),
    'B3': mean.get('B3'),
    'B4': mean.get('B4'),
    'B5': mean.get('B5'),
    'B6': mean.get('B6'),
    'B7': mean.get('B7'),
    'B8': mean.get('B8'),
    'B8A': mean.get('B8A'),
    'B11': mean.get('B11'),
    'B12': mean.get('B12'),
    'date': image.get('system:time_start')
  });
};

// Map the function over the image collection
var spectralSignatures = s2Collection.map(getSpectralSignature);

// Convert to a feature collection
var spectralFC = ee.FeatureCollection(spectralSignatures);

// Chart the spectral signatures
var chart = ui.Chart.feature.byFeature(spectralFC, 'date', ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])
               .setChartType('LineChart')
               .setOptions({
                 title: 'Spectral Signature',
                 vAxis: {title: 'Reflectance'},
                 hAxis: {title: 'Date'},
                 series: {0: {color: 'blue'}, 1: {color: 'green'}, 2: {color: 'red'}, 3: {color: 'orange'}, 
                           4: {color: 'purple'}, 5: {color: 'cyan'}, 6: {color: 'brown'}, 7: {color: 'pink'},
                           8: {color: 'magenta'}, 9: {color: 'black'}},
                 legend: {position: 'bottom'},
                 pointSize: 3
               });

// Add the chart to the map
print(chart);


SyntaxError: invalid syntax (1515963824.py, line 1)

In [20]:
pip install gdal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.6/840.6 kB 1.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [86 lines of output]
      Using numpy 2.1.2
      running egg_info
      writing gdal-utils/GDAL.egg-info/PKG-INFO
      writing dependency_links to gdal-utils/GDAL.egg-info/dependency_links.txt
      writing entry points to gdal-utils/GDAL.egg-info/entry_points.txt
      writing requirements to gdal-utils/GDAL.egg-info/requires.txt
      writing top-level names to gdal-utils/GDAL.egg-info/top_level.txt
      Traceback (most recent call last):
        File "<string>", line 91, in fetch_config
        File "/Users/poojyanthm/anaconda3/lib/python3.10/subprocess.py", line 971, in __init__
          self._execute_child(args, executable, preexec_fn, close_fds,
        File "/Users/pooj

In [22]:
pip install osgeo

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [36 lines of output]
      running bdist_wheel
      running build
      /Users/poojyanthm/anaconda3/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      installing to build/bdist.macosx-11.1-arm64/wheel
      running install
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/c1/ctjwbtwn47ndwfwg_ss4my8m0000gn/T/pip-install-pzbt09pj/osgeo_3ff009a5a96e43a49f716c1cfddda67e/setup.py", line 32, in <module>
          setup(
        File "/Users/poojyanthm/anaconda3/lib/python3.10/site-packages/setuptools/__init__.py", line 87, in setup
          return d

In [1]:
import os
import pandas as pd
import numpy as np
from osgeo import gdal
from pyproj import Transformer

def parse_mtl_file(mtl_path):
    """Parse the MTL metadata file for the Landsat scene."""
    params = {}
    with open(mtl_path, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.split('=')
                params[key.strip()] = value.strip().strip('"')
    return params

def filter_landsat_scenes(directory, cloud_threshold=15, date_range=None):
    """Filter Landsat scenes based on cloud cover and optional date range."""
    filtered_scenes = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('_MTL.txt'):
                mtl_path = os.path.join(root, file)
                metadata = parse_mtl_file(mtl_path)
                
                # Extract cloud cover and acquisition date
                cloud_cover = float(metadata.get('CLOUD_COVER', 100))
                acquisition_date = metadata.get('DATE_ACQUIRED', '')
                
                # Check if scene meets cloud cover criteria
                if cloud_cover <= cloud_threshold:
                    if date_range:
                        if date_range[0] <= acquisition_date <= date_range[1]:
                            filtered_scenes.append(mtl_path)
                    else:
                        filtered_scenes.append(mtl_path)
    
    return filtered_scenes

ModuleNotFoundError: No module named 'osgeo'

In [2]:
pip install rasterio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 1.6 MB/s eta 0:00:0000:0100:01m
  Using cached numpy-2.1.2-cp310-cp310-macosx_14_0_arm64.whl (5.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
scipy 1.10.0 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.1.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 2.1.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
